# HackDavis OSIsoft OCS Jupyter Notebook 

![](https://apimgmtstelkv30lahnuj362.blob.core.windows.net/content/MediaLibrary/hackdavis/hackdavis_header.png)

## OCS-HackDavis Package Installation 

In [3]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ ocs-hackdavis==0.34.0

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/


## Import functions from package `ocs_hackdavis`

### They are all described in this notebook with examples

In [4]:
from ocs_hackdavis import (
    ucdavis_buildings,  # list of campus buildings
    ucdavis_ceeds_of,   # list of CEED element of a building (Electricity, Steam, Chilled Water, etc)
    ucdavis_streams_of, # The list of all OCS data streams for a building and CEED pair  
    ucdavis_building_metadata,  # Metadata for a building: building code, lat/long, usage, etc.  
    ocs_stream_interpolated_data,  # Interpolated data from a stream given a time range + interpolation interval
    ucdavis_outside_temperature,  # Outside temperature at UC Davis for a given a time range + interpolation interval
)

## List of buildings

In [5]:
buildings = ucdavis_buildings()
len(buildings), buildings[:25]  # display first 25

(159,
 ['ARC Pavilion',
  'Academic Surge Building',
  'Activities and Recreation Center',
  'Advanced Materials Research Laboratory',
  'Advanced Transportation Infrastructure Research Center',
  'Aggie Stadium',
  'Agronomy Field Laboratory',
  'Animal Building',
  'Animal Resource Service J1',
  'Animal Resource Service M3',
  'Animal Resource Service N1',
  'Ann E. Pitzer Center',
  'Antique Mechanics Trailer',
  'Aquatic Biology & Environmental Science Bldg',
  'Art Building',
  'Art Building Annex',
  'Art, Music, Wright Halls',
  'Asmundson Annex',
  'Asmundson Hall',
  'Bainer Hall',
  'Bowley Head House',
  'Briggs Hall',
  'California Hall',
  'Campus Data Center',
  'Cellular Biology Laboratory'])

## Function `ucdavis_ceeds_of` returns list of CEEDs for a given building

### Display them for all buildings

In [6]:
for num, building in enumerate(ucdavis_buildings(), 1):
    print(f"[{num}] Building: {building} ==> CEEDS: {ucdavis_ceeds_of(building)}")

[1] Building: ARC Pavilion ==> CEEDS: ['ChilledWater', 'Electricity', 'Steam']
[2] Building: Academic Surge Building ==> CEEDS: ['ChilledWater', 'Electricity', 'Steam']
[3] Building: Activities and Recreation Center ==> CEEDS: ['ChilledWater', 'Electricity', 'Steam']
[4] Building: Advanced Materials Research Laboratory ==> CEEDS: ['Electricity']
[5] Building: Advanced Transportation Infrastructure Research Center ==> CEEDS: ['Electricity']
[6] Building: Aggie Stadium ==> CEEDS: ['Electricity']
[7] Building: Agronomy Field Laboratory ==> CEEDS: ['Electricity']
[8] Building: Animal Building ==> CEEDS: ['Electricity']
[9] Building: Animal Resource Service J1 ==> CEEDS: ['Electricity', 'Natural Gas']
[10] Building: Animal Resource Service M3 ==> CEEDS: ['Electricity']
[11] Building: Animal Resource Service N1 ==> CEEDS: ['Electricity']
[12] Building: Ann E. Pitzer Center ==> CEEDS: ['ChilledWater', 'Electricity', 'Steam']
[13] Building: Antique Mechanics Trailer ==> CEEDS: ['Electricity']


## Get the list of available streams of a building

#### Default CEED is Electricity

In [17]:
ucdavis_streams_of("Kemper Hall")

{'AnnualUsage': 'PI_uni-pida-vm0_859',
 'Cumulative Use': 'PI_uni-pida-vm0_838',
 'Demand': 'PI_uni-pida-vm0_883',
 'Demand_kBtu': 'PI_uni-pida-vm0_860',
 'Electricity_EUI': 'PI_uni-pida-vm0_895',
 'MonthlyUsage': 'PI_uni-pida-vm0_858',
 'Rollover Check': 'PI_uni-pida-vm0_896',
 'Rollover Count Month': 'PI_uni-pida-vm0_897',
 'Rollover Count Year': 'PI_uni-pida-vm0_898'}

In [47]:
ucdavis_streams_of("Young Hall")

{'AnnualCost': 'PI_uni-pida-vm0_11690',
 'AnnualUsage': 'PI_uni-pida-vm0_2164',
 'Cumulative Use': 'PI_uni-pida-vm0_2162',
 'Demand': 'PI_uni-pida-vm0_2300_ds2m',
 'Demand_kBtu': 'PI_uni-pida-vm0_2165',
 'EUI': 'PI_uni-pida-vm0_2314',
 'Electricity_EUI': 'PI_uni-pida-vm0_2369',
 'MonthlyUsage': 'PI_uni-pida-vm0_2163',
 'Rollover Check': 'PI_uni-pida-vm0_2370',
 'Rollover Count Month': 'PI_uni-pida-vm0_2371',
 'Rollover Count Year': 'PI_uni-pida-vm0_2372'}

## Get the specific stream of a building

In [40]:
ucdavis_streams_of("Kemper Hall", "chilledWater")["Demand"]

#warning: Building name "Kemper Hall" and/or CEED "chilledWater" are not valid


KeyError: 

## Import for OCS Sample Library and other necessary modules

In [14]:
from ocs_sample_library_preview import OCSClient
import configparser
import io
import json
import pandas as pd

## Standard configuration file parsing and OCS client object 

#### Reference: https://github.com/osisoft/OSI-Samples-OCS/blob/master/basic_samples/DataViews/Python3/program.py#L150

## IMPORTANT: REPLACE CLIENT ID + SECRET BELOW BEFORE RUNNING - ASK OSISOFT PERSONEL FOR URL

In [15]:
config_text = u"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = UC__Davis

[Access]
Resource = https://dat-b.osisoft.com
Tenant = 65292b6c-ec16-414a-b583-ce7ae04046d4
ApiVersion = v1-preview

[Credentials] 
ClientId = 82fca0c2-3004-42c0-81cf-cc6968df1f47
ClientSecret = 3VYy318vxlFGKVuQ4+toahSyg7IqWUVKiGHJDvH/IvY=
"""

In [16]:
config = configparser.ConfigParser(allow_no_value=True)
config.read_file(io.StringIO(config_text))

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

namespace_id = config.get("Configurations", "Namespace")
print(f"namespace_id: '{namespace_id}'")

namespace_id: 'UC__Davis'


## Getting interpolated data from a stream
'AnnualUsage': 'PI_uni-pida-vm0_859',
 'Cumulative Use': 'PI_uni-pida-vm0_838',
 'Demand': 'PI_uni-pida-vm0_883',
 'Demand_kBtu': 'PI_uni-pida-vm0_860',
 'Electricity_EUI': 'PI_uni-pida-vm0_895',
 'MonthlyUsage': 'PI_uni-pida-vm0_858',
 'Rollover Check': 'PI_uni-pida-vm0_896',
 'Rollover Count Month': 'PI_uni-pida-vm0_897',
 'Rollover Count Year': 'PI_uni-pida-vm0_898'

In [34]:
# Step 1: get the stream Id 
stream_id = ucdavis_streams_of("Kemper Hall")["Rollover Count Year"]

# Step 2) request interpolated data
# NOTE 1: difference between endIndex and startIndex should be 31 days or less
# NOTE 2: interpolation interval cannot be less than 2 minutes
result1 = ocs_stream_interpolated_data(
    ocs_client,
    namespace_id,
    stream_id,
    start="2017-02-01", # UTC 
    end="2017-03-01",
    interval=60*24,  # 2 minutes
)

result1[:25]

[{'Timestamp': '2017-02-01T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-02T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-03T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-04T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-05T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-06T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-07T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-08T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-09T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-10T00:00:00Z', 'Value': 1.0},
 {'Timestamp': '2017-02-11T00:00:00Z', 'Value': 0.333333343},
 {'Timestamp': '2017-02-12T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-13T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-14T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-15T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-16T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-17T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017-02-18T00:00:00Z', 'Value': 0.0},
 {'Timestamp': '2017

## Transform result into a Pandas series

In [35]:
demand1 = pd.read_json(json.dumps(result)).set_index('Timestamp')
demand1

,Value
Timestamp,
2017-01-01 08:00:00+00:00,1
2017-01-01 08:02:00+00:00,1
2017-01-01 08:04:00+00:00,1
2017-01-01 08:06:00+00:00,1
2017-01-01 08:08:00+00:00,1
...,...
2017-01-31 23:52:00+00:00,1
2017-01-31 23:54:00+00:00,1
2017-01-31 23:56:00+00:00,1


## Get data for next month

In [36]:
result = ocs_stream_interpolated_data(
    ocs_client,
    namespace_id,
    stream_id,
    start="2017-01-01", # UTC 
    end="2017-2-1",
    interval=2,  # 2 minutes
)

## Transform into a Pandas time series and append to first month data

In [37]:
demand2 = pd.read_json(json.dumps(result)).set_index('Timestamp')
demand_2m = demand1.append(demand2)
len(demand_2m), demand_2m

(44162,                            Value
 Timestamp                       
 2017-01-01 08:00:00+00:00      1
 2017-01-01 08:02:00+00:00      1
 2017-01-01 08:04:00+00:00      1
 2017-01-01 08:06:00+00:00      1
 2017-01-01 08:08:00+00:00      1
 ...                          ...
 2017-01-31 23:52:00+00:00      1
 2017-01-31 23:54:00+00:00      1
 2017-01-31 23:56:00+00:00      1
 2017-01-31 23:58:00+00:00      1
 2017-02-01 00:00:00+00:00      1
 
 [44162 rows x 1 columns])

## There is a repeated row at the boundary of the two appended series

More specifically, the bottom row of first series and first row of second series. Line below shows it:

In [22]:
demand_2m.loc[demand_2m.index.duplicated()]

,Value
Timestamp,
2017-03-01 00:00:00+00:00,258.662231


## Remove duplicated row

Difference in lenght (== # of rows) should be 1 

In [23]:
new_demand = demand_2m.loc[~demand_2m.index.duplicated(keep="first")]
len(demand_2m) - len(new_demand)

1

## Getting a full year of data

### Reusing what we've learn so far 

In [24]:
# Create an empty series
demand = pd.Series()
demand

Series([], dtype: float64)

In [25]:
for start_month in range(1, 12):
    start_date = f"2017-{start_month}-01"
    print(f"> processing {start_date}")
    result = ocs_stream_interpolated_data(
        ocs_client,
        namespace_id,
        stream_id,
        start=start_date,  # UTC
        end=f"2017-{start_month+1}-01",
        interval=2,  # 2 minutes
    )
    demand = demand.append(
        pd.read_json(json.dumps(result)).set_index("Timestamp", drop=True)
    )

# Note there is an additional column "0" with NaN (not a number), it will be remove in the next cell
len(demand), demand

> processing 2017-1-01
> processing 2017-2-01


/home/mostafa_shokrof/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


> processing 2017-3-01
> processing 2017-4-01
> processing 2017-5-01
> processing 2017-6-01
> processing 2017-7-01
> processing 2017-8-01
> processing 2017-9-01
> processing 2017-10-01
> processing 2017-11-01


(240250,                             0       Value
 2017-01-01 08:02:00+00:00 NaN  203.602783
 2017-01-01 08:04:00+00:00 NaN  196.223251
 2017-01-01 08:06:00+00:00 NaN  195.362305
 2017-01-01 08:08:00+00:00 NaN  195.213913
 2017-01-01 08:10:00+00:00 NaN  197.648666
 ...                        ..         ...
 2017-11-30 23:52:00+00:00 NaN  228.839342
 2017-11-30 23:54:00+00:00 NaN  228.842742
 2017-11-30 23:56:00+00:00 NaN  228.846142
 2017-11-30 23:58:00+00:00 NaN  228.849542
 2017-12-01 00:00:00+00:00 NaN  228.852943
 
 [240250 rows x 2 columns])

In [26]:
# remove duplicate rows and keep only column named "Values"
demand = demand.loc[~demand.index.duplicated(keep="first")]["Value"]
len(demand), demand

(240240, 2017-01-01 08:02:00+00:00    203.602783
 2017-01-01 08:04:00+00:00    196.223251
 2017-01-01 08:06:00+00:00    195.362305
 2017-01-01 08:08:00+00:00    195.213913
 2017-01-01 08:10:00+00:00    197.648666
                                 ...    
 2017-11-30 23:52:00+00:00    228.839342
 2017-11-30 23:54:00+00:00    228.842742
 2017-11-30 23:56:00+00:00    228.846142
 2017-11-30 23:58:00+00:00    228.849542
 2017-12-01 00:00:00+00:00    228.852943
 Name: Value, Length: 240240, dtype: float64)

## Building metadata

In [27]:
ucdavis_building_metadata(ocs_client, namespace_id, "Activities and Recreation Center")

{'Annual Cost': 151156.0,
 'BuildingName': 'ARC',
 'kWh Rate': 0.0687,
 'Prefix': 'Activities_and_Recreation_Center_MSB',
 'Rollover': 10000000.0,
 'CAAN': 4799.0,
 'Construction Date': '04/15/2002',
 'Display Name': 'Activities and Recreation Center',
 'Latitude': 38.5428969596,
 'Longitude': -121.759644393,
 'Maintained Gross Sq. Ft.': 158120.0,
 'Primary Usage (Type)': 'REC - Athletics & Recreation'}

## Get outside temperature at UC Davis

### Data for this stream starts on 2017-04-01, 19:38

In [28]:
# display only first 100 rows
ucdavis_outside_temperature(ocs_client, namespace_id, "2017-04-01", "2017-05-01", 2)[:100]

[{'Timestamp': '2017-04-02T19:38:00Z', 'Value': 73.84828},
 {'Timestamp': '2017-04-02T19:40:00Z', 'Value': 74.04701},
 {'Timestamp': '2017-04-02T19:42:00Z', 'Value': 74.13842},
 {'Timestamp': '2017-04-02T19:44:00Z', 'Value': 74.0657959},
 {'Timestamp': '2017-04-02T19:46:00Z', 'Value': 74.32633},
 {'Timestamp': '2017-04-02T19:48:00Z', 'Value': 74.61136},
 {'Timestamp': '2017-04-02T19:50:00Z', 'Value': 74.52496},
 {'Timestamp': '2017-04-02T19:52:00Z', 'Value': 74.7417145},
 {'Timestamp': '2017-04-02T19:54:00Z', 'Value': 75.06158},
 {'Timestamp': '2017-04-02T19:56:00Z', 'Value': 75.67546},
 {'Timestamp': '2017-04-02T19:58:00Z', 'Value': 75.66438},
 {'Timestamp': '2017-04-02T20:00:00Z', 'Value': 75.3251953},
 {'Timestamp': '2017-04-02T20:02:00Z', 'Value': 75.10666},
 {'Timestamp': '2017-04-02T20:04:00Z', 'Value': 75.3027649},
 {'Timestamp': '2017-04-02T20:06:00Z', 'Value': 75.17714},
 {'Timestamp': '2017-04-02T20:08:00Z', 'Value': 75.3776245},
 {'Timestamp': '2017-04-02T20:10:00Z', 'Value'

## Plotting data with Plotly Express

**Plotly Express (https://plot.ly/python/plotly-express/) is a very good plotting library with easy path to web app deployment with its DASH (https://plot.ly/dash/) companion**

In [29]:
!pip install plotly

     |████████████████████████████████| 7.3MB 3.3MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11429 sha256=c3d8a903edbc7fdf74b70310cc9a5023545dde714ba1ddc70ab70456fda5741e
  Stored in directory: /home/mostafa_shokrof/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [30]:
import plotly.express as px 

## Select two days of demand data and transform into tidy dataframe format

In [31]:
df = demand['2017-03-01':'2017-03-02'].to_frame().reset_index()
df.columns = ['Time', 'Demand']
df

,Time,Demand
0,2017-03-01 00:00:00+00:00,258.662231
1,2017-03-01 00:02:00+00:00,261.630829
2,2017-03-01 00:04:00+00:00,247.635773
3,2017-03-01 00:06:00+00:00,252.180191
4,2017-03-01 00:08:00+00:00,248.544693
...,...,...
1435,2017-03-02 23:50:00+00:00,252.279266
1436,2017-03-02 23:52:00+00:00,253.621201
1437,2017-03-02 23:54:00+00:00,265.473969
1438,2017-03-02 23:56:00+00:00,259.356354


## Graph 

In [45]:
fig = px.line(df, x="Time", y="Demand")
fig.show()

### For more info on tidy format and other visualization libraries

Tidy: https://tomaugspurger.github.io/modern-5-tidy.html

Visualization: https://tomaugspurger.github.io/modern-6-visualization

## Test error code 

### Bad argument of the right type

In [33]:
# should fail
ucdavis_ceeds_of("Bad Building")

#warning: "Bad Building" is not a valid building name
